In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import scipy
from skspatial.objects import LineSegment, Line
from enum import Enum

In [ ]:
VIA_DIAM = 0.8
VIA_DRILL = 0.4
STATOR_HOLE_RADIUS = 4
TRACK_WIDTH = 0.2
TRACK_SPACING = 0.2
TURNS = 11
STATOR_RADIUS = 20
Layer = Enum("Layer", "FRONT BACK")

In [ ]:
template = [(-1.5, -0.1), (1.5, -2), (2.0, -1), (2.0, 1), (1.5, 2), (-1.5, 0.1)]

In [ ]:
# plot the template shape wrapping around to the first point
plt.pyplot.plot(
    [x for x, y in template] + [template[0][0]],
    [y for x, y in template] + [template[0][1]],
)

In [ ]:
# this speeds things up dramatically as we only have to compute the line intersections once
# there are probably much faster ways of doing this - we're just doing a brute force search
# for the intersections - consult algorithms from games for inspiration...
def get_template_point_cache(template):
    # sweep a line from the origin through 360 degress times the number of turns in 1 degree increments
    # and find the intersection points with the template shape
    cache = {}
    for angle in np.arange(0, 360 + 1, 1):
        line = LineSegment(
            np.array([0, 0]),
            np.array(
                [1000 * np.cos(np.deg2rad(angle)), 1000 * np.sin(np.deg2rad(angle))]
            ),
        )
        for i in range(len(template)):
            segment = LineSegment(
                np.array(template[i]), np.array(template[(i + 1) % len(template)])
            )
            try:
                intersection = line.intersect_line_segment(segment)
                if intersection is not None:
                    cache[angle] = (intersection, segment)
            except ValueError:
                pass
    return cache


def get_point(angle, template, layer, spacing, cache):
    if layer == Layer.BACK:
        angle = angle + 180
    intersection, segment = cache[angle % 360]
    return intersection, segment


# get the points in a coil shape
# Use reverse for bottom layer (basically flips the y coordinate so that the coil goes in the opposite direction)
# Also rotates the endpoints by 90 degress so that the exit point on the bottom layer is to the left hand side
def get_points(template, turns, spacing, layer=Layer.FRONT, cache=None):
    if cache is None:
        cache = get_template_point_cache(template)
    coil_points = []
    last_segment = None
    for angle in np.arange(0, 360 * turns + 1, 1):
        offset = spacing * angle / 360
        intersection, segment = get_point(angle, template, layer, spacing, cache)
        vector = np.array(segment.point_a) - np.array(segment.point_b)
        normal = vector / np.linalg.norm(vector)
        # rotate the vector 90 degrees
        normal = np.array([-normal[1], normal[0]])
        # move the intersection point along the normal vector by the spacing
        coil_point = intersection + normal * offset
        if layer == Layer.BACK:
            coil_points.append((coil_point[0], -coil_point[1], segment))
        else:
            coil_points.append((coil_point[0], coil_point[1], segment))
    # run through the generated coil points and where the line segments change add a point that is the intersection of the previous and next lines
    # this prevents any corner cutting
    points = []
    last_segment = coil_points[0][2]
    for i in range(len(coil_points)):
        x, y, segment = coil_points[i]
        same_segment = (
            (last_segment.point_a == segment.point_a).all()
            and (last_segment.point_b == segment.point_b).all()
        ).all()
        if (not same_segment) and i > 2 and i < len(coil_points) - 2:
            # create a line from the previous two points
            line = Line(
                np.array(coil_points[i - 2][0:2]),
                np.array(coil_points[i - 2][0:2]) - np.array(coil_points[i - 1][0:2]),
            )
            # create a line from the next two points
            line2 = Line(
                np.array(coil_points[i][0:2]),
                np.array(coil_points[i][0:2]) - np.array(coil_points[i + 1][0:2]),
            )
            # find the intersection of the two lines
            intersection = line.intersect_line(line2)
            # add the intersection point to the list of points
            points.append(intersection)
        last_segment = segment
        points.append((x, y))
    return points


def optimize_points(points):
    # follow the line and remove points that are in the same direction as the previous poin
    # keep doing this until the direction changes significantly
    # this is a very simple optimization that removes a lot of points
    # it's not perfect but it's a good start
    optimized_points = []
    for i in range(len(points)):
        if i == 0:
            optimized_points.append(points[i])
        else:
            vector1 = np.array(points[i]) - np.array(points[i - 1])
            vector2 = np.array(points[(i + 1) % len(points)]) - np.array(points[i])
            length1 = np.linalg.norm(vector1)
            length2 = np.linalg.norm(vector2)
            if length1 > 0 and length2 > 0:
                dot = np.dot(vector1, vector2) / (length1 * length2)
                # clamp dot between -1 and 1
                dot = max(-1, min(1, dot))
                angle = np.arccos(dot)
                if angle > np.deg2rad(0.1):
                    optimized_points.append(points[i])
    print("Optimised from {} to {} points".format(len(points), len(optimized_points)))
    return optimized_points


def chaikin(points, iterations):
    if iterations == 0:
        return points
    l = len(points)
    smoothed = []
    for i in range(l - 1):
        x1, y1 = points[i]
        x2, y2 = points[i + 1]
        smoothed.append([0.9 * x1 + 0.1 * x2, 0.9 * y1 + 0.1 * y2])
        smoothed.append([0.1 * x1 + 0.9 * x2, 0.1 * y1 + 0.9 * y2])
    smoothed.append(points[l - 1])
    return chaikin(smoothed, iterations - 1)

In [ ]:
points = get_points(template, 10, TRACK_SPACING + TRACK_WIDTH, Layer.BACK, None)
optimized_points = chaikin(optimize_points(points), 3)
# df = pd.DataFrame(points, columns=['x', 'y'])
# ax = df.plot.line(x='x', y='y', label='Coil A', color='blue')
# ax.axis('equal')

df_optim = pd.DataFrame(optimized_points, columns=["x", "y"])

ax = df_optim.plot.line(x="x", y="y", label="Coil A", color="green")
ax.axis("equal")
print(len(df_optim), len(df))

In [ ]:
cache = get_template_point_cache(template)

In [ ]:
points_f = get_points(template, TURNS, TRACK_SPACING + TRACK_WIDTH, Layer.FRONT, cache)
points_b = get_points(template, TURNS, TRACK_SPACING + TRACK_WIDTH, Layer.BACK, cache)

points_f = [(0, 0)] + chaikin(optimize_points(points_f), 3)
points_b = [(0, 0)] + chaikin(optimize_points(points_b), 3)

COIL_CENTER_RADIUS = STATOR_RADIUS / 2 + 1.5

angle_A = 0
angle_B = 120
angle_C = 240

# roate the points by the required angle
def rotate(points, angle):
    return [
        [
            x * np.cos(np.deg2rad(angle)) - y * np.sin(np.deg2rad(angle)),
            x * np.sin(np.deg2rad(angle)) + y * np.cos(np.deg2rad(angle)),
        ]
        for x, y in points
    ]


# move the points out to the distance at the requited angle
def translate(points, distance, angle):
    return [
        [
            x + distance * np.cos(np.deg2rad(angle)),
            y + distance * np.sin(np.deg2rad(angle)),
        ]
        for x, y in points
    ]


# flip the y coordinate
def flip(points):
    return [[x, -y] for x, y in points]


# the main coils
coil_A_f = translate(rotate(points_f, angle_A), COIL_CENTER_RADIUS, angle_A)
coil_A_b = translate(rotate(points_b, angle_A), COIL_CENTER_RADIUS, angle_A)

coil_B_f = translate(rotate(points_f, angle_B), COIL_CENTER_RADIUS, angle_B)
coil_B_b = translate(rotate(points_b, angle_B), COIL_CENTER_RADIUS, angle_B)

coil_C_f = translate(rotate(points_f, angle_C), COIL_CENTER_RADIUS, angle_C)
coil_C_b = translate(rotate(points_b, angle_C), COIL_CENTER_RADIUS, angle_C)

# the opposite coils - for more power!
angle_A_opp = angle_A + 180
angle_B_opp = angle_B + 180
angle_C_opp = angle_C + 180

print(angle_A_opp, angle_B_opp, angle_C_opp)

coil_A_opp_f = translate(
    rotate(flip(points_f), angle_A_opp), COIL_CENTER_RADIUS, angle_A_opp
)
coil_A_opp_b = translate(
    rotate(flip(points_b), angle_A_opp), COIL_CENTER_RADIUS, angle_A_opp
)

coil_B_opp_f = translate(
    rotate(flip(points_f), angle_B_opp), COIL_CENTER_RADIUS, angle_B_opp
)
coil_B_opp_b = translate(
    rotate(flip(points_b), angle_B_opp), COIL_CENTER_RADIUS, angle_B_opp
)

coil_C_opp_f = translate(
    rotate(flip(points_f), angle_C_opp), COIL_CENTER_RADIUS, angle_C_opp
)
coil_C_opp_b = translate(
    rotate(flip(points_b), angle_C_opp), COIL_CENTER_RADIUS, angle_C_opp
)

# connect the front copper opposite coils together
common_connection_radius = STATOR_RADIUS - (TRACK_WIDTH + TRACK_SPACING)
common_coil_connections_b = [
    (
        common_connection_radius * np.cos(np.deg2rad(angle)),
        common_connection_radius * np.sin(np.deg2rad(angle)),
    )
    for angle in np.arange(angle_A_opp, angle_C_opp + 5, 5)
]
coil_A_opp_f.append(
    (
        common_connection_radius * np.cos(np.deg2rad(angle_A_opp)),
        common_connection_radius * np.sin(np.deg2rad(angle_A_opp)),
    )
)
coil_B_opp_f.append(
    (
        common_connection_radius * np.cos(np.deg2rad(angle_B_opp)),
        common_connection_radius * np.sin(np.deg2rad(angle_B_opp)),
    )
)
coil_C_opp_f.append(
    (
        common_connection_radius * np.cos(np.deg2rad(angle_C_opp)),
        common_connection_radius * np.sin(np.deg2rad(angle_C_opp)),
    )
)

# connect coil A to it's opposite
connection_radius1 = STATOR_HOLE_RADIUS + (TRACK_SPACING)
connection_radius2 = connection_radius1 + (TRACK_SPACING + VIA_DIAM / 2)
# draw a 45 degree line from coil A at connection radius 1
# then connect up to connection radius 2
# draw a 45 degree line to the opposite coil
coil_A_b.append(
    (
        connection_radius1 * np.cos(np.deg2rad(angle_A)),
        connection_radius1 * np.sin(np.deg2rad(angle_A)),
    )
)
coil_A_opp_b.append(
    (
        connection_radius2 * np.cos(np.deg2rad(angle_A_opp)),
        connection_radius2 * np.sin(np.deg2rad(angle_A_opp)),
    )
)
a_connection_b = [
    (
        connection_radius1 * np.cos(np.deg2rad(angle)),
        connection_radius1 * np.sin(np.deg2rad(angle)),
    )
    for angle in np.arange(angle_A, angle_A + 90 + 5, 5)
]
a_connection_f = [
    (
        connection_radius2 * np.cos(np.deg2rad(angle)),
        connection_radius2 * np.sin(np.deg2rad(angle)),
    )
    for angle in np.arange(angle_A + 90, angle_A + 180 + 5, 5)
]
a_connection_b.append(a_connection_f[0])

coil_B_b.append(
    (
        connection_radius1 * np.cos(np.deg2rad(angle_B)),
        connection_radius1 * np.sin(np.deg2rad(angle_B)),
    )
)
coil_B_opp_b.append(
    (
        connection_radius2 * np.cos(np.deg2rad(angle_B_opp)),
        connection_radius2 * np.sin(np.deg2rad(angle_B_opp)),
    )
)
b_connection_b = [
    (
        connection_radius1 * np.cos(np.deg2rad(angle)),
        connection_radius1 * np.sin(np.deg2rad(angle)),
    )
    for angle in np.arange(angle_B, angle_B + 90 + 5, 5)
]
b_connection_f = [
    (
        connection_radius2 * np.cos(np.deg2rad(angle)),
        connection_radius2 * np.sin(np.deg2rad(angle)),
    )
    for angle in np.arange(angle_B + 90, angle_B + 180 + 5, 5)
]
b_connection_b.append(b_connection_f[0])

coil_C_b.append(
    (
        connection_radius1 * np.cos(np.deg2rad(angle_C)),
        connection_radius1 * np.sin(np.deg2rad(angle_C)),
    )
)
coil_C_opp_b.append(
    (
        connection_radius2 * np.cos(np.deg2rad(angle_C_opp)),
        connection_radius2 * np.sin(np.deg2rad(angle_C_opp)),
    )
)
c_connection_b = [
    (
        connection_radius1 * np.cos(np.deg2rad(angle)),
        connection_radius1 * np.sin(np.deg2rad(angle)),
    )
    for angle in np.arange(angle_C, angle_C + 90 + 5, 5)
]
c_connection_f = [
    (
        connection_radius2 * np.cos(np.deg2rad(angle)),
        connection_radius2 * np.sin(np.deg2rad(angle)),
    )
    for angle in np.arange(angle_C + 90, angle_C + 180 + 5, 5)
]
c_connection_b.append(c_connection_f[0])

In [ ]:
def create_track(points):
    return [{"x": x, "y": y} for x, y in points]

In [ ]:
# dump out the results to json
json_result = {
    "parameters": {
        "trackWidth": TRACK_WIDTH,
        "statorHoleRadius": STATOR_HOLE_RADIUS,
        "viaDiameter": VIA_DIAM,
        "viaDrillDiameter": VIA_DRILL,
    },
    "vias": [
        {
            "x": COIL_CENTER_RADIUS * np.cos(np.deg2rad(angle_A)),
            "y": COIL_CENTER_RADIUS * np.sin(np.deg2rad(angle_A)),
        },
        {
            "x": COIL_CENTER_RADIUS * np.cos(np.deg2rad(angle_B)),
            "y": COIL_CENTER_RADIUS * np.sin(np.deg2rad(angle_B)),
        },
        {
            "x": COIL_CENTER_RADIUS * np.cos(np.deg2rad(angle_C)),
            "y": COIL_CENTER_RADIUS * np.sin(np.deg2rad(angle_C)),
        },
        {
            "x": COIL_CENTER_RADIUS * np.cos(np.deg2rad(angle_A_opp)),
            "y": COIL_CENTER_RADIUS * np.sin(np.deg2rad(angle_A_opp)),
        },
        {
            "x": COIL_CENTER_RADIUS * np.cos(np.deg2rad(angle_B_opp)),
            "y": COIL_CENTER_RADIUS * np.sin(np.deg2rad(angle_B_opp)),
        },
        {
            "x": COIL_CENTER_RADIUS * np.cos(np.deg2rad(angle_C_opp)),
            "y": COIL_CENTER_RADIUS * np.sin(np.deg2rad(angle_C_opp)),
        },
        {
            "x": common_connection_radius * np.cos(np.deg2rad(angle_A_opp)),
            "y": common_connection_radius * np.sin(np.deg2rad(angle_A_opp)),
        },
        {
            "x": common_connection_radius * np.cos(np.deg2rad(angle_B_opp)),
            "y": common_connection_radius * np.sin(np.deg2rad(angle_B_opp)),
        },
        {
            "x": common_connection_radius * np.cos(np.deg2rad(angle_C_opp)),
            "y": common_connection_radius * np.sin(np.deg2rad(angle_C_opp)),
        },
        # coil A connections
        {"x": a_connection_f[0][0], "y": a_connection_f[0][1]},
        {"x": a_connection_f[-1][0], "y": a_connection_f[-1][1]},
        # coil B connections
        {"x": b_connection_f[0][0], "y": b_connection_f[0][1]},
        {"x": b_connection_f[-1][0], "y": b_connection_f[-1][1]},
        # coil C connections
        {"x": c_connection_f[0][0], "y": c_connection_f[0][1]},
        {"x": c_connection_f[-1][0], "y": c_connection_f[-1][1]},
    ],
    "silk": [
        {
            "x": COIL_CENTER_RADIUS * np.cos(np.deg2rad(angle_A)),
            "y": COIL_CENTER_RADIUS * np.sin(np.deg2rad(angle_A)),
            "text": "A",
        },
        {
            "x": COIL_CENTER_RADIUS * np.cos(np.deg2rad(angle_B)),
            "y": COIL_CENTER_RADIUS * np.sin(np.deg2rad(angle_B)),
            "text": "B",
        },
        {
            "x": COIL_CENTER_RADIUS * np.cos(np.deg2rad(angle_C)),
            "y": COIL_CENTER_RADIUS * np.sin(np.deg2rad(angle_C)),
            "text": "C",
        },
    ],
    "tracks": {
        "f": [
            create_track(points)
            for points in [
                coil_A_f,
                coil_A_opp_f,
                coil_B_f,
                coil_B_opp_f,
                coil_C_f,
                coil_C_opp_f,
                a_connection_f,
                b_connection_f,
                c_connection_f,
            ]
        ],
        "b": [
            create_track(points)
            for points in [
                coil_A_b,
                coil_A_opp_b,
                coil_B_b,
                coil_B_opp_b,
                coil_C_b,
                coil_C_opp_b,
                common_coil_connections_b,
                a_connection_b,
                b_connection_b,
                c_connection_b,
            ]
        ],
    },
}

import json

json.dump(json_result, open("coil.json", "w"))


df = pd.DataFrame(coil_A_f, columns=["x", "y"])
ax = df.plot.line(x="x", y="y", label="Coil A", color="blue")
ax.axis("equal")
df = pd.DataFrame(coil_A_b, columns=["x", "y"])
ax = df.plot.line(x="x", y="y", label="Coil B", color="green")
ax.axis("equal")

# plot all three coils on the same graph
# df = pd.DataFrame(coil_A, columns=['x', 'y'])
# ax = df.plot.line(x='x', y='y', label='Coil A', color='blue')
# ax.axis('equal')
# df = pd.DataFrame(coil_B, columns=['x', 'y'])
# df.plot.line(x='x', y='y', ax=ax, label='Coil B', color='green')
# df = pd.DataFrame(coil_C, columns=['x', 'y'])
# df.plot.line(x='x', y='y', ax=ax, label='Coil C', color='red')

# df = pd.DataFrame(coil_A_opposite, columns=['x', 'y'])
# df.plot.line(x='x', y='y', ax=ax, label='Coil A Opposite', color='blue')
# df = pd.DataFrame(coil_B_opposite, columns=['x', 'y'])
# df.plot.line(x='x', y='y', ax=ax, label='Coil B Opposite', color='green')
# df = pd.DataFrame(coil_C_opposite, columns=['x', 'y'])
# df.plot.line(x='x', y='y', ax=ax, label='Coil C Opposite', color='red')